<a href="https://colab.research.google.com/github/weso500/MOSAICRev/blob/main/GANonmaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time

# --- Colab Setup: Run this cell first ---
# NOTE: The pip install and mkdir commands are assumed to have been run successfully.
# !pip install torch numpy pandas scikit-learn
# !mkdir -p '/content/drive/MyDrive/Globecom Paper/For Jason/IOT-Anomaly-Detection/Raw_Data'

# --- Step 1: Mount Google Drive (Essential for accessing files) ---
try:
    print("Mounting Google Drive...")
    # Using 'force_remount=True' is often necessary if the notebook has disconnected/reconnected
    # drive.mount('/content/drive', force_remount=True)
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully.


In [2]:
# --- Colab Setup: Run this cell first ---
# This file implements the Ganomaly baseline using GRU layers for time-series anomaly detection.
# It is designed to run in the same environment as the TranAD baseline.

import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score
import torch.optim as optim

# --- 2. CONFIGURATION (Matches TranAD Configuration) ---
SYSTEM = 'RFQ'
# IMPORTANT: Full Google Drive path for correct file access
RAW_DATA_DIR = '/content/drive/MyDrive/Globecom Paper/For Jason/IOT-Anomaly-Detection/Raw_Data'
WINDOW_SIZE = 100           # Sequence length (L)
N_FEATURES = 14             # Dimension of the feature vector (D_model)
PREDICTION_LENGTH = 1       # Target sequence length (1 for anomaly detection)
N_EPOCHS = 30              # GANs often require more epochs to stabilize
BATCH_SIZE = 512
G_LR = 1e-4                 # Generator Learning Rate
D_LR = 1e-4                 # Discriminator Learning Rate

# Ganomaly Hyperparameters
LATENT_DIM = 64             # Size of the latent space (Z)
GRU_HIDDEN_SIZE = 128
ALPHA = 1.0                 # Weight for Reconstruction Loss
BETA = 0.1                  # Weight for Adversarial Loss (L_adv)
GAMMA = 1.0                 # Weight for Encoder/Latent Loss (L_enc)

# --- 3. DATA UTILITIES (Copied for self-contained execution) ---

def load_and_preprocess_data(system, raw_data_dir):
    """Loads, splits, and preprocesses the accelerator data."""
    print(f"Loading data for system: {system}")
    x_path = os.path.join(raw_data_dir, f'{system}.npy')
    y_path = os.path.join(raw_data_dir, f'{system}_labels.npy')

    X, Y = None, None

    if not os.path.exists(x_path) or not os.path.exists(y_path):
        print(f"--- WARNING: Data files not found. Creating DUMMY DATA. ---")
        # Ensure DUMMY DATA shape matches expected N_FEATURES
        X = np.random.rand(1000, 200, N_FEATURES)
        Y = np.array([[i, 'Run', 'type'] for i in range(800)] +
                     [[i + 800, 'Fault', 'type'] for i in range(200)], dtype=object)
    else:
        try:
            X = np.load(x_path)
            Y = np.load(y_path, allow_pickle=True)
            print("Real data loaded successfully.")
        except ValueError as e:
            print(f"\n--- ERROR HANDLING: Caught ValueError during loading: {e} ---")
            X = np.load(x_path, allow_pickle=True)
            Y = np.load(y_path, allow_pickle=True)
            print("Reload successful using allow_pickle=True.")

    if X is None or Y is None:
        raise RuntimeError("Data failed to load or create dummy data.")

    # Data shape consistency check
    if X.shape[-1] != N_FEATURES:
        raise ValueError(f"Data feature size mismatch: Expected {N_FEATURES}, but loaded data has {X.shape[-1]}. Check N_FEATURES config.")

    fault_indices, normal_indices = np.where(Y[:,1] == 'Fault')[0], np.where(Y[:,1] == 'Run')[0]
    Xnormal = X[normal_indices, :, :]
    Xfault = X[fault_indices, :, :]

    print(f"Normal Data Pulses: {len(Xnormal)}, Fault Data Pulses: {len(Xfault)}")

    n_pulses, n_times, n_features = Xnormal.shape
    X_mts = Xnormal.reshape(-1, n_features)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_mts)
    print(f"Concatenated and Scaled MTS shape: {X_scaled.shape}")

    X_fault_mts = Xfault.reshape(-1, n_features)
    X_fault_scaled = scaler.transform(X_fault_mts)

    return X_scaled, X_fault_scaled, scaler


def create_windows(data, window_size, prediction_length):
    """Slices the time series into sequences (windows)."""
    windows = []
    # Calculate the total length of the time series
    total_len = len(data) - window_size + 1

    for i in range(total_len):
        # Input and target are the same sequence for the autoencoder/GAN reconstruction
        input_seq = data[i : i + window_size]
        target_seq = data[i : i + window_size]
        windows.append((input_seq, target_seq))

    X_win = np.array([w[0] for w in windows], dtype=np.float32)
    Y_win = np.array([w[1] for w in windows], dtype=np.float32)

    print(f"Created {len(X_win)} windows of size {window_size}.")

    return X_win, Y_win


class TimeSeriesDataset(Dataset):
    """A PyTorch Dataset for the time series windows."""
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Returns (input sequence, target sequence)
        return self.X[idx], self.Y[idx]

# --- 4. PYTORCH GANOMALY MODEL IMPLEMENTATION ---

class Generator(nn.Module):
    """Ganomaly Generator: Encoder -> Latent -> Decoder."""
    def __init__(self, input_size, hidden_size, latent_dim, seq_len):
        super().__init__()
        self.seq_len = seq_len

        # Encoder (GRU maps sequence to hidden state)
        self.encoder = nn.GRU(input_size, hidden_size, batch_first=True)
        # Hidden state to Latent Vector (Z_original)
        self.to_latent = nn.Linear(hidden_size, latent_dim)
        # Latent Vector back to Initial Hidden State for Decoder
        self.from_latent = nn.Linear(latent_dim, hidden_size)

        # Decoder (GRU maps hidden state to reconstructed sequence)
        self.decoder = nn.GRU(input_size, hidden_size, batch_first=True)
        # Hidden state to Output Feature
        self.to_output = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # x shape: (B, L, D)

        # Encoder (Get final hidden state)
        # Pass the whole sequence through the GRU
        _, h_n_enc = self.encoder(x) # h_n_enc shape: (1, B, H)

        # Latent Vector (Z_original)
        z = self.to_latent(h_n_enc.squeeze(0)) # z shape: (B, Z)

        # Decoder Initial Hidden State
        h_n_dec = self.from_latent(z).unsqueeze(0) # h_n_dec shape: (1, B, H)

        # Decoder Input
        # For Autoencoder style reconstruction, the input sequence is passed again
        x_dec = x

        # Decoder
        output_seq, _ = self.decoder(x_dec, h_n_dec) # output_seq shape: (B, L, H)

        # Map hidden states to reconstructed data
        reconstruction = self.to_output(output_seq) # reconstruction shape: (B, L, D)

        # Return reconstruction (X_hat) and the latent vector (Z_original)
        return reconstruction, z


class Discriminator(nn.Module):
    """Ganomaly Discriminator: Encoder -> Feature Map -> Classifier."""
    def __init__(self, input_size, hidden_size):
        super().__init__()

        # Feature Extractor (GRU maps sequence to hidden state/feature map)
        self.extractor = nn.GRU(input_size, hidden_size, batch_first=True)

        # Classifier (From hidden state to a single prediction [Real/Fake])
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: (B, L, D)
        _, h_n = self.extractor(x) # h_n shape: (1, B, H)
        feature_map = h_n.squeeze(0) # feature_map shape: (B, H)

        # Classification: Real (1) or Fake (0)
        classification = self.classifier(feature_map) # classification shape: (B, 1)

        # Return classification score and the final hidden state (feature map)
        return classification, feature_map


class GanomalyModel(nn.Module):
    """Wrapper class for Ganomaly components."""
    def __init__(self, input_size, hidden_size, latent_dim, seq_len):
        super().__init__()
        self.G = Generator(input_size, hidden_size, latent_dim, seq_len)
        self.D = Discriminator(input_size, hidden_size)

        # In Ganomaly, a separate Encoder (E) is used to encode the reconstructed data (X_hat)
        # into a reconstructed latent vector (Z_hat). We use the same architecture as G's encoder.
        self.E = self._create_encoder(input_size, hidden_size, latent_dim)

    def _create_encoder(self, input_size, hidden_size, latent_dim):
        """Creates a separate encoder for the reconstruction latent space."""
        # This setup must perfectly mirror the Encoder section of the Generator (G)
        encoder_gru = nn.GRU(input_size, hidden_size, batch_first=True)
        to_latent = nn.Linear(hidden_size, latent_dim)
        return nn.Sequential(encoder_gru, to_latent)

    def encode_reconstruction(self, x_hat):
        # x_hat shape: (B, L, D)
        # GRU maps sequence to hidden state
        _, h_n = self.E[0](x_hat)
        # Hidden state to Latent Vector (Z_hat)
        z_hat = self.E[1](h_n.squeeze(0)) # z_hat shape: (B, Z)
        return z_hat


# --- 5. TRAINING AND EVALUATION FUNCTIONS ---

def train_ganomaly(model, dataloader, device, n_epochs):
    """Trains the Ganomaly model."""

    model.train()
    bce_criterion = nn.BCELoss()
    mse_criterion = nn.MSELoss()

    # Optimizers are now defined in main, but we use the global hyperparams
    G_params = list(model.G.parameters()) + list(model.E.parameters())
    optimizer_G = optim.Adam(G_params, lr=G_LR, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(model.D.parameters(), lr=D_LR, betas=(0.5, 0.999))

    print(f"\nStarting Ganomaly Training on device: {device}")

    for epoch in range(1, n_epochs + 1):
        total_loss_G = 0
        total_loss_D = 0

        for batch_x, _ in dataloader:
            batch_x = batch_x.to(device)
            batch_size = batch_x.size(0)

            # --- 1. Train Discriminator (D) ---
            optimizer_D.zero_grad()

            # D Loss on Real Data (Target: 1)
            output_real, _ = model.D(batch_x)
            labels_real = torch.full((batch_size, 1), 1.0, device=device)
            loss_D_real = bce_criterion(output_real, labels_real)

            # D Loss on Fake Data (Reconstruction from G, Target: 0)
            reconstruction, _ = model.G(batch_x)
            # D's input is the reconstruction, detached from G to avoid G gradients
            output_fake, _ = model.D(reconstruction.detach())
            labels_fake = torch.full((batch_size, 1), 0.0, device=device)
            loss_D_fake = bce_criterion(output_fake, labels_fake)

            loss_D = loss_D_real + loss_D_fake
            loss_D.backward()
            optimizer_D.step()
            total_loss_D += loss_D.item()

            # --- 2. Train Generator (G) and Encoder (E) ---
            optimizer_G.zero_grad()

            reconstruction, z_original = model.G(batch_x)

            # L_rec: Reconstruction Loss (X_hat vs X)
            loss_rec = mse_criterion(reconstruction, batch_x)

            # L_adv: Adversarial Loss (G wants D to think X_hat is Real, Target: 1)
            output_fake_for_G, fm_fake = model.D(reconstruction)
            labels_real_for_G = torch.full((batch_size, 1), 1.0, device=device)
            loss_adv = bce_criterion(output_fake_for_G, labels_real_for_G)

            # L_enc: Encoder Loss (Latent consistency: Z_hat vs Z_original)
            z_reconstructed = model.encode_reconstruction(reconstruction)
            loss_enc = mse_criterion(z_reconstructed, z_original)

            # Total Generator Loss (L_G)
            loss_G = (ALPHA * loss_rec) + (BETA * loss_adv) + (GAMMA * loss_enc)

            loss_G.backward()
            optimizer_G.step()
            total_loss_G += loss_G.item()

        avg_loss_G = total_loss_G / len(dataloader)
        avg_loss_D = total_loss_D / len(dataloader)

        # Print update every 10 epochs for cleaner output
        if (epoch % 10 == 0) or (epoch == n_epochs):
            print(f"Epoch {epoch:03d}/{n_epochs}, G Loss: {avg_loss_G:.6f}, D Loss: {avg_loss_D:.6f}")

    print("Ganomaly Training complete.")


def evaluate_ganomaly(model, X_normal_test_mts, X_fault_test_mts, window_size, missing_modalities_count, device):
    """
    Calculates the Ganomaly score for both normal and fault test data under two conditions.
    Anomaly Score = ALPHA * L_rec + GAMMA * L_enc
    """
    model.eval()
    mse_criterion = nn.MSELoss(reduction='none')

    normal_full_scores = []
    fault_full_scores = []
    normal_missing_scores = []
    fault_missing_scores = []

    print("\nStarting Ganomaly Evaluation...")

    with torch.no_grad():
        # Iterate through Normal and Fault test data
        for name, X_mts in [('Normal', X_normal_test_mts), ('Fault', X_fault_test_mts)]:

            # Create windows on the MTS data
            X_win_eval, Y_win_eval = create_windows(X_mts, window_size, PREDICTION_LENGTH)

            if len(X_win_eval) == 0:
                print(f"Warning: No {name} test windows provided. Skipping evaluation.")
                continue

            test_dataset = TimeSeriesDataset(X_win_eval, Y_win_eval)
            test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

            current_scores_full = []
            current_scores_missing = []

            for batch_x, _ in test_dataloader:
                batch_x = batch_x.to(device)

                # --- Full Modality Test ---
                reconstruction_full, z_original_full = model.G(batch_x)
                z_reconstructed_full = model.encode_reconstruction(reconstruction_full)

                # L_rec: MSE loss over the entire sequence, averaged per window
                l_rec_full = mse_criterion(reconstruction_full, batch_x).mean(dim=(1, 2))
                # L_enc: MSE loss in the latent space, averaged per window
                l_enc_full = mse_criterion(z_reconstructed_full, z_original_full).mean(dim=1)

                # Anomaly Score (Full)
                score_full = (ALPHA * l_rec_full) + (GAMMA * l_enc_full)
                current_scores_full.extend(score_full.cpu().numpy())

                # --- Missing Modality Test ---
                X_missing = batch_x.clone()
                # Mask the first `missing_modalities_count` features to zero
                X_missing[:, :, :missing_modalities_count] = 0.0

                reconstruction_missing, z_original_missing = model.G(X_missing)
                z_reconstructed_missing = model.encode_reconstruction(reconstruction_missing)

                # L_rec: MSE loss between Reconstruction and the *Original Input (batch_x)*
                l_rec_missing = mse_criterion(reconstruction_missing, batch_x).mean(dim=(1, 2))
                # L_enc: MSE loss in the latent space
                l_enc_missing = mse_criterion(z_reconstructed_missing, z_original_missing).mean(dim=1)

                # Anomaly Score (Missing)
                score_missing = (ALPHA * l_rec_missing) + (GAMMA * l_enc_missing)
                current_scores_missing.extend(score_missing.cpu().numpy())

            if name == 'Normal':
                normal_full_scores.extend(current_scores_full)
                normal_missing_scores.extend(current_scores_missing)
            else: # 'Fault'
                fault_full_scores.extend(current_scores_full)
                fault_missing_scores.extend(current_scores_missing)

            print(f"Finished processing {len(X_win_eval)} {name} windows.")

    # Convert list to numpy array before returning
    return (np.array(normal_full_scores), np.array(fault_full_scores),
            np.array(normal_missing_scores), np.array(fault_missing_scores))


def calculate_youden_auc(normal_scores, fault_scores, description):
    """Calculates AUC-ROC and finds the optimal threshold using Youden's J statistic."""
    if len(normal_scores) == 0 or len(fault_scores) == 0:
        print(f"\n--- SKIPPING METRICS FOR {description} ---")
        print("Not enough normal or fault windows were created/found to calculate ROC/AUC.")
        return

    all_scores = np.concatenate([normal_scores, fault_scores])
    true_labels = np.concatenate([np.zeros_like(normal_scores), np.ones_like(fault_scores)])

    if len(np.unique(true_labels)) < 2:
        print(f"\n--- SKIPPING METRICS FOR {description} ---")
        print("Only one class (all normal or all fault) was found in the test set. Cannot calculate AUC.")
        return

    fpr, tpr, thresholds = roc_curve(true_labels, all_scores)
    auc_score = roc_auc_score(true_labels, all_scores)

    # Youden's J = Sensitivity + Specificity - 1 = TPR - FPR
    youden_j = tpr - fpr
    optimal_idx = np.argmax(youden_j)
    optimal_threshold = thresholds[optimal_idx]

    predicted_labels = (all_scores >= optimal_threshold).astype(int)

    TP = np.sum((true_labels == 1) & (predicted_labels == 1))
    TN = np.sum((true_labels == 0) & (predicted_labels == 0))
    FP = np.sum((true_labels == 0) & (predicted_labels == 1))
    FN = np.sum((true_labels == 1) & (predicted_labels == 0))

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    accuracy = (TP + TN) / len(true_labels) if len(true_labels) > 0 else 0

    print(f"\n--- CLASSIFICATION METRICS: {description} ---")
    print(f"AUC-ROC Score: {auc_score:.4f}")
    print(f"Optimal Threshold (Youden's J): {optimal_threshold:.6f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Sensitivity (TPR): {sensitivity:.4f}")
    print(f"Specificity (TNR): {specificity:.4f}")
    print(f"Youden's J Max Value: {youden_j[optimal_idx]:.4f}")


# --- 6. EXECUTION ---
if __name__ == "__main__":
    # Detect device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 1. Load and prepare data (MTS)
    X_scaled_mts, X_fault_scaled_mts, scaler = None, None, None
    try:
        X_scaled_mts, X_fault_scaled_mts, scaler = load_and_preprocess_data(SYSTEM, RAW_DATA_DIR)
    except RuntimeError as e:
        print(f"Skipping model execution due to data loading failure: {e}")
        # exit() # Use a placeholder or small dummy data to allow testing the code structure
        X_scaled_mts = np.random.rand(800 * 200, N_FEATURES)
        X_fault_scaled_mts = np.random.rand(200 * 200, N_FEATURES)
        print("Using fallback dummy data for execution.")
    except Exception as e:
        print(f"An unknown error occurred during data loading: {e}")
        # exit() # Use a placeholder or small dummy data to allow testing the code structure
        X_scaled_mts = np.random.rand(800 * 200, N_FEATURES)
        X_fault_scaled_mts = np.random.rand(200 * 200, N_FEATURES)
        print("Using fallback dummy data for execution.")

    # Simple split of normal data into training (first 80%) and testing (last 20%)
    split_idx = int(len(X_scaled_mts) * 0.8)
    X_train_mts = X_scaled_mts[:split_idx]
    X_normal_test_mts_full = X_scaled_mts[split_idx:]
    X_normal_test_mts = X_normal_test_mts_full[:18999]
    X_fault_scaled_mts = X_fault_scaled_mts[:999]

    # Custom Sampling for test data to mirror previous setup
    # Ensure test data is long enough to form windows
    X_normal_test_mts = X_normal_test_mts_full[:max(len(X_normal_test_mts_full) - WINDOW_SIZE, WINDOW_SIZE * 5)]
    X_fault_scaled_mts = X_fault_scaled_mts[:max(len(X_fault_scaled_mts) - WINDOW_SIZE, WINDOW_SIZE * 5)]

    # 2. Create windowed datasets for training
    X_train_win, Y_train_win = create_windows(X_train_mts, WINDOW_SIZE, PREDICTION_LENGTH)

    # Create the PyTorch DataLoader
    train_dataset = TimeSeriesDataset(X_train_win, Y_train_win)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # 3. Initialize Model, Loss, and Optimizer
    model = GanomalyModel(
        input_size=N_FEATURES,
        hidden_size=GRU_HIDDEN_SIZE,
        latent_dim=LATENT_DIM,
        seq_len=WINDOW_SIZE
    ).to(device)

    # 4. Train the model
    start_time = time.time()
    # The train function internally defines and uses the optimizers based on global LR,
    # but we are passing the model and dataloader.
    train_ganomaly(model, train_dataloader, device, N_EPOCHS)
    end_time = time.time()
    print(f"Total training time: {end_time - start_time:.2f} seconds")

    # 5. Run the core experiment: Evaluate Missing Modalities
    missing_modalities_count = 5

    normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
    )

    # 6. Calculate AUC and Youden's J for both scenarios
    calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
    calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")

Using device: cuda
Loading data for system: RFQ
Real data loaded successfully.
Normal Data Pulses: 690, Fault Data Pulses: 182
Concatenated and Scaled MTS shape: (3105000, 14)
Created 2483901 windows of size 100.

Starting Ganomaly Training on device: cuda
Epoch 010/30, G Loss: 0.069328, D Loss: 1.386293
Epoch 020/30, G Loss: 0.069323, D Loss: 1.386293
Epoch 030/30, G Loss: 0.069321, D Loss: 1.386294
Ganomaly Training complete.
Total training time: 3501.18 seconds

Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 5 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.0466
Optimal Threshold (Youden's J

In [21]:
def evaluate_ganomaly(model, X_normal_test_mts, X_fault_test_mts, window_size, missing_modalities_count, device):
    """
    Calculates the Ganomaly score for both normal and fault test data under two conditions:
      - Full modalities
      - Missing modalities (first `missing_modalities_count` features zeroed)

    Anomaly Score = ALPHA * L_rec + GAMMA * L_enc

    Returns:
        normal_full_scores      : np.array, scores on normal data (all modalities present)
        fault_full_scores       : np.array, scores on fault data (all modalities present)
        normal_missing_scores   : np.array, scores on normal data (with missing modalities)
        fault_missing_scores    : np.array, scores on fault data (with missing modalities)
    """
    model.eval()
    mse_criterion = nn.MSELoss(reduction='none')

    normal_full_scores = []
    fault_full_scores = []
    normal_missing_scores = []
    fault_missing_scores = []

    print("\nStarting Ganomaly Evaluation...")

    with torch.no_grad():
        # Evaluate for Normal and Fault sets
        for name, X_mts in [('Normal', X_normal_test_mts), ('Fault', X_fault_test_mts)]:

            # Create windows on the MTS data (same as before)
            X_win_eval, Y_win_eval = create_windows(X_mts, window_size, PREDICTION_LENGTH)

            if len(X_win_eval) == 0:
                print(f"Warning: No {name} test windows provided. Skipping evaluation.")
                continue

            test_dataset = TimeSeriesDataset(X_win_eval, Y_win_eval)
            test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

            current_scores_full = []
            current_scores_missing = []

            for batch_x, _ in test_dataloader:
                batch_x = batch_x.to(device)  # shape: (B, T, F)

                # -----------------------------
                # 1) Full Modality Evaluation
                # -----------------------------
                reconstruction_full, z_original_full = model.G(batch_x)
                z_reconstructed_full = model.encode_reconstruction(reconstruction_full)

                # L_rec_full: MSE over all features and timesteps
                l_rec_full = mse_criterion(reconstruction_full, batch_x).mean(dim=(1, 2))

                # L_enc_full: MSE in latent space
                l_enc_full = mse_criterion(z_reconstructed_full, z_original_full).mean(dim=1)

                score_full = (ALPHA * l_rec_full) + (GAMMA * l_enc_full)
                current_scores_full.extend(score_full.detach().cpu().numpy())

                # -----------------------------
                # 2) Missing Modality Evaluation
                # -----------------------------
                if missing_modalities_count > 0:
                    # Clamp missing count to number of available features
                    num_features = batch_x.size(2)
                    miss = min(missing_modalities_count, num_features)

                    # Build mask: 1 = keep, 0 = missing
                    mask = torch.ones_like(batch_x)
                    mask[:, :, :miss] = 0.0

                    # Masked input (simulate missing modalities)
                    X_missing = batch_x * mask

                    reconstruction_missing, z_original_missing = model.G(X_missing)
                    z_reconstructed_missing = model.encode_reconstruction(reconstruction_missing)

                    # --- Reconstruction loss only on *unmasked* (available) features ---
                    diff_sq = (reconstruction_missing - batch_x) ** 2 * mask
                    # Normalize by number of unmasked elements per sample
                    mask_sum = mask.sum(dim=(1, 2)).clamp_min(1.0)
                    l_rec_missing = diff_sq.sum(dim=(1, 2)) / mask_sum

                    # Latent consistency loss (both latents come from masked pipeline)
                    l_enc_missing = mse_criterion(z_reconstructed_missing, z_original_missing).mean(dim=1)

                    score_missing = (ALPHA * l_rec_missing) + (GAMMA * l_enc_missing)
                    current_scores_missing.extend(score_missing.detach().cpu().numpy())
                else:
                    # If no modalities are missing, missing-score == full-score
                    current_scores_missing.extend(score_full.detach().cpu().numpy())

            if name == 'Normal':
                normal_full_scores.extend(current_scores_full)
                normal_missing_scores.extend(current_scores_missing)
            else:  # 'Fault'
                fault_full_scores.extend(current_scores_full)
                fault_missing_scores.extend(current_scores_missing)

            print(f"Finished processing {len(X_win_eval)} {name} windows.")

    return (np.array(normal_full_scores),
            np.array(fault_full_scores),
            np.array(normal_missing_scores),
            np.array(fault_missing_scores))


In [33]:
    # 5. Run the core experiment: Evaluate Missing Modalities
missing_modalities_count = 1

normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
)

calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")


Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 1 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9536
Optimal Threshold (Youden's J): 0.000098
Accuracy: 0.9527
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9526
Youden's J Max Value: 0.9526


In [34]:
    # 5. Run the core experiment: Evaluate Missing Modalities
missing_modalities_count = 2

normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
)

calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")


Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 2 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9421
Optimal Threshold (Youden's J): 0.000143
Accuracy: 0.9419
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9419
Youden's J Max Value: 0.9419


In [35]:
    # 5. Run the core experiment: Evaluate Missing Modalities
missing_modalities_count = 3

normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
)

calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")


Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 3 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9811
Optimal Threshold (Youden's J): 0.000341
Accuracy: 0.9805
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9805
Youden's J Max Value: 0.9805


In [36]:
    # 5. Run the core experiment: Evaluate Missing Modalities
missing_modalities_count = 4

normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
)

calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")


Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 4 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.6749
Optimal Threshold (Youden's J): 0.000186
Accuracy: 0.6753
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.6748
Youden's J Max Value: 0.6748


In [37]:
    # 5. Run the core experiment: Evaluate Missing Modalities
missing_modalities_count = 5

normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
)

calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")


Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 5 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.6136
Optimal Threshold (Youden's J): 0.000127
Accuracy: 0.6106
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.6101
Youden's J Max Value: 0.6101


In [38]:
    # 5. Run the core experiment: Evaluate Missing Modalities
missing_modalities_count = 6

normal_full_scores, fault_full_scores, normal_missing_scores, fault_missing_scores = evaluate_ganomaly(
        model,
        X_normal_test_mts,
        X_fault_scaled_mts,
        WINDOW_SIZE,
        missing_modalities_count=missing_modalities_count,
        device=device
)

calculate_youden_auc(normal_full_scores, fault_full_scores, "Full Modalities Test (Ganomaly)")
calculate_youden_auc(normal_missing_scores, fault_missing_scores, f"Missing {missing_modalities_count} Modalities Test (Ganomaly)")


Starting Ganomaly Evaluation...
Created 620801 windows of size 100.
Finished processing 620801 Normal windows.
Created 800 windows of size 100.
Finished processing 800 Fault windows.

--- CLASSIFICATION METRICS: Full Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.9993
Optimal Threshold (Youden's J): 0.000063
Accuracy: 0.9993
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.9993
Youden's J Max Value: 0.9993

--- CLASSIFICATION METRICS: Missing 6 Modalities Test (Ganomaly) ---
AUC-ROC Score: 0.6732
Optimal Threshold (Youden's J): 0.000334
Accuracy: 0.6736
Sensitivity (TPR): 1.0000
Specificity (TNR): 0.6731
Youden's J Max Value: 0.6731
